In [1]:
from typing import List

import gensim
from tqdm import tqdm

In [2]:
%%time
model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin', binary=True)

CPU times: user 22.2 s, sys: 3.14 s, total: 25.3 s
Wall time: 27.9 s


In [3]:
filename = 'data/questions-words.txt'
dataset = dict()
data = []
key = None
with open(filename, 'rt') as f:
    for line in f:
        words = line.strip()
        if words.startswith(':'):
            if len(data) > 0 and key is not None:
                dataset[key] = data
                data = []
            key = words.split()[1]
        else:
            data.append(words.split())
print(dataset.keys())

dict_keys(['capital-common-countries', 'capital-world', 'currency', 'city-in-state', 'family', 'gram1-adjective-to-adverb', 'gram2-opposite', 'gram3-comparative', 'gram4-superlative', 'gram5-present-participle', 'gram6-nationality-adjective', 'gram7-past-tense', 'gram8-plural'])


In [4]:
target_keys = [key for key in dataset.keys() if key.startswith('gram')]
scores = []
for key in tqdm(target_keys):
    for data in dataset[key]:
        result = model.most_similar(positive=[data[1], data[2]], negative=[data[0]], topn=1)
        pred_word = result[0][0]
        scores.append(pred_word == data[3])
accuracy_score = len(list(filter(lambda x: x, scores))) / len(scores)
print(f'semantic accuracy: {accuracy_score}')

100%|██████████| 8/8 [59:49<00:00, 448.66s/it]semantic accuracy: 0.7454360020397757



In [ ]:
target_keys = [key for key in dataset.keys() if not key.startswith('gram')]
scores = []
for key in tqdm(target_keys):
    for data in dataset[key]:
        result = model.most_similar(positive=[data[1], data[2]], negative=[data[0]], topn=1)
        pred_word = result[0][0]
        scores.append(pred_word == data[3])
accuracy_score = len(list(filter(lambda x: x, scores))) / len(scores)
print(f'syntatic accuracy: {accuracy_score}')